# Hoja de Trabajo \# 4

---


por Josué Obregón <br>
DS6011 - Feature Engineering <br>
UVG Masters - Escuela de Negocios<br>


## Objetivos

El objetivo de esta hoja de trabajo  es presentar al estudiante diferentes técnicas de codificación de variables categóricas. Éstas técnicas incluyen codificadores clásicos, de contraste y codificadores supervisados o bayesianos.

También se busca que el estudiante practique la utilización de éstas técnicas con las librerías disponibles en el lenguaje Python.


## Importación de librerías y carga de los datos a varios pandas [DataFrames](https://pandas.pydata.org/pandas-docs/version/1.1.5/reference/frame.html)




In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [2]:
!mkdir data

In [3]:
import gdown

urls = ['https://drive.google.com/uc?export=download&id=16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq', # okc_train  https://drive.google.com/file/d/16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq/view?usp=sharing
        ]
outputs = ['okc_train.csv']
for url,output in zip(urls,outputs):
  gdown.download(url, f'data/{output}', quiet=False)

Downloading...
From: https://drive.google.com/uc?export=download&id=16AGQw1nM9NYILv2aSZaSNSn9jBPByWPq
To: /content/data/okc_train.csv
12.3MB [00:00, 80.5MB/s]


In [4]:
df = pd.read_csv('data/okc_train.csv',index_col=0)

In [5]:
df.head()

,age,diet,drinks,drugs,education,height,income,last_online,offspring,pets,religion,sign,smokes,status,where_state,where_town,religion_modifer,sign_modifer,cpp,cpp_fluently,cpp_okay,cpp_poorly,lisp,lisp_fluently,lisp_okay,lisp_poorly,asian,black,hispanic_latin,indian,middle_eastern,native_american,other,pacific_islander,white,essay_length,Class,profile
1,22,strictly_anything,socially,never,working_on_college_university,75,missing,3,doesnt_have_kids_but_might_want_them,likes_dogs_and_likes_cats,agnosticism,gemini,sometimes,single,california,south_san_francisco,and_very_serious_about_it,sign_mod_missing,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,3.420945,other,1
4,23,vegetarian,socially,drugs_missing,working_on_college_university,71,inc20000,3,doesnt_want_kids,likes_cats,religion_missing,pisces,no,single,california,berkeley,religion_mod_missing,sign_mod_missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.671173,other,3
6,29,mostly_anything,socially,drugs_missing,graduated_from_college_university,67,missing,2,doesnt_have_kids_but_might_want_them,likes_cats,atheism,taurus,no,single,california,san_francisco,religion_mod_missing,sign_mod_missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.391112,stem,5
8,31,mostly_anything,socially,never,graduated_from_college_university,65,missing,2,doesnt_have_kids_but_wants_them,likes_dogs_and_likes_cats,christianity,sagittarius,no,single,california,san_francisco,religion_mod_missing,sign_mod_missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.090611,other,6
10,37,mostly_anything,not_at_all,never,working_on_two_year_college,65,missing,3,kids_missing,likes_dogs_and_likes_cats,atheism,cancer,no,single,california,san_mateo,and_laughing_about_it,but_it_doesnt_matter,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3.382377,other,7


In [6]:
df['drinks'].value_counts()

socially          27521
rarely             3981
often              3413
not_at_all         2157
drinks_missing     1208
very_often          319
desperately         210
Name: drinks, dtype: int64

In [7]:
df['status'].value_counts()

single            36014
seeing_someone     1354
available          1234
married             204
unknown               3
Name: status, dtype: int64

Pequeño dataset de prueba para algunas explicaciones y para las figuras en las diapositivas


In [8]:
df_test = pd.DataFrame({
... 'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC',
... 'Seattle', 'Seattle', 'Seattle'],
... 'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]
... })

In [9]:
df_test

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


Para esta hoja de trabajo vamos a utilizar la librería [category_encoders](http://contrib.scikit-learn.org/category_encoders/index.html), la cual es compatible con scikit-learn.

In [10]:
!pip install category_encoders

     |████████████████████████████████| 81kB 6.6MB/s 


In [11]:
col = 'drinks'

# Unsupervised Encoders

## Classic Encoders

### One-Hot Encoder

In [12]:
from category_encoders import OneHotEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [13]:
onehot_enc = OneHotEncoder(use_cat_names=True)

In [14]:
onehot_enc.fit_transform(df[[col]])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



,drinks_socially,drinks_not_at_all,drinks_rarely,drinks_often,drinks_drinks_missing,drinks_very_often,drinks_desperately
1,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0
8,1,0,0,0,0,0,0
10,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
59940,1,0,0,0,0,0,0
59943,0,0,0,1,0,0,0
59944,0,1,0,0,0,0,0
59945,1,0,0,0,0,0,0


In [15]:
onehot_enc.mapping[0]['mapping']

,drinks_socially,drinks_not_at_all,drinks_rarely,drinks_often,drinks_drinks_missing,drinks_very_often,drinks_desperately
1,1,0,0,0,0,0,0
2,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0
5,0,0,0,0,1,0,0
6,0,0,0,0,0,1,0
7,0,0,0,0,0,0,1
-1,0,0,0,0,0,0,0
-2,0,0,0,0,0,0,0


### Binary Encoder

In [16]:
from category_encoders import BinaryEncoder

In [17]:
binary_enc = BinaryEncoder()

In [18]:
binary_enc.fit_transform(df[[col]])

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead



,drinks_0,drinks_1,drinks_2,drinks_3
1,0,0,0,1
4,0,0,0,1
6,0,0,0,1
8,0,0,0,1
10,0,0,1,0
...,...,...,...,...
59940,0,0,0,1
59943,0,1,0,0
59944,0,0,1,0
59945,0,0,0,1


In [19]:
binary_enc.base_n_encoder.mapping[0]['mapping']

,drinks_0,drinks_1,drinks_2,drinks_3
1,0,0,0,1
2,0,0,1,0
3,0,0,1,1
4,0,1,0,0
5,0,1,0,1
6,0,1,1,0
7,0,1,1,1
-1,0,0,0,0
-2,0,0,0,0


### Frequency or Count Encoder

In [20]:
from category_encoders import CountEncoder

In [21]:
count_enc = CountEncoder(normalize=False)

In [22]:
df[col].value_counts()

socially          27521
rarely             3981
often              3413
not_at_all         2157
drinks_missing     1208
very_often          319
desperately         210
Name: drinks, dtype: int64

Intentemos utilizando las funciones de combinacion de grupos

In [23]:
count_enc = CountEncoder(combine_min_nan_groups=True, min_group_size=500, min_group_name='otros', normalize=True)

### Ordinal Encoder

In [24]:
from category_encoders import OrdinalEncoder

In [25]:
ord_enc = OrdinalEncoder()

In [26]:
ord_enc.fit_transform(df[col])

,drinks
1,1
4,1
6,1
8,1
10,2
...,...
59940,1
59943,4
59944,2
59945,1


In [27]:
ord_enc.category_mapping[0]['mapping']

socially          1
not_at_all        2
rarely            3
often             4
drinks_missing    5
very_often        6
desperately       7
NaN              -2
dtype: int64

In [28]:
drink_dict = {None:0, 'not_at_all': 1, 'reraly':2,
              'socially':3,'often':4,'very_often':5 ,
              'desperately':6, 'drinks_missing':-1}
col_drink_dict = {'col':'drinks','mapping':drink_dict}
ord_enc = OrdinalEncoder(mapping = [col_drink_dict])
ord_enc.fit_transform(df[col],)

,drinks
1,3.0
4,3.0
6,3.0
8,3.0
10,1.0
...,...
59940,3.0
59943,4.0
59944,1.0
59945,3.0


###Feature Hashing

In [29]:
from sklearn.feature_extraction import FeatureHasher

In [30]:
col = 'where_town'

In [31]:
df[col].describe()

count             38809
unique               51
top       san_francisco
freq              20028
Name: where_town, dtype: object

In [32]:
hash_enc = FeatureHasher(n_features=8, input_type='string')

In [33]:
hashed_features = hash_enc.fit_transform(df[col])

In [34]:
hashed_features.toarray()

array([[ 1., -1.,  2., ..., -2.,  0., -3.],
       [ 0.,  1.,  0., ..., -1.,  1.,  2.],
       [ 0., -1.,  2., ..., -1.,  0., -2.],
       ...,
       [ 1., -1.,  2., ..., -2.,  0., -3.],
       [ 0., -1.,  2., ..., -1.,  0., -2.],
       [ 0., -1.,  2., ..., -1.,  0., -2.]])

In [35]:
np_hashed = np.array(hashed_features.todense())

In [36]:
np.unique(np_hashed, axis=0)

array([[ 0., -2.,  2.,  1.,  0., -2., -1.,  0.],
       [ 0., -1.,  0.,  4.,  0., -1.,  1.,  1.],
       [ 0., -1.,  1.,  0., -1., -3.,  0.,  1.],
       [ 0., -1.,  1.,  1.,  1., -1.,  2.,  0.],
       [ 0., -1.,  2.,  3., -2., -1.,  0., -2.],
       [ 0.,  0.,  0.,  0.,  1., -1.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0., -1., -1.,  1.],
       [ 0.,  0.,  0.,  2.,  0., -2.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  0., -1.,  0.,  1.],
       [ 0.,  0.,  1.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  2.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  5., -1.,  1.,  1.],
       [ 0.,  0.,  1.,  3., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  2., -1.,  1.,  0.,  1.,  2.],
       [ 0.,  0.,  2.,  0.,  0., -1.,  1.,  0.],
       [ 0.,  0.,  2.,  0.,  2., -2., -1.,  0.],
       [ 0.,  0.,  2.,  0.,  2.,  0.,  1.,  0.],
       [ 0.,  0.,  2.,  1.,  0., -1.,  1.,  0.],
       [ 0.,  0.,  2.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  2.,  2.,  0., -1., -1., -1.],
       [ 0.,  0.,  2

In [37]:
np.unique(np_hashed,axis=0).shape

(50, 8)

In [38]:
from sys import getsizeof

In [39]:
print('Our pandas Series, in bytes: ', getsizeof(df[col]))
print('Our hashed numpy array, in bytes: ', getsizeof(hashed_features))

Our pandas Series, in bytes:  2944494
Our hashed numpy array, in bytes:  64


## Contrast Encoders

### Diferencia entre One-hot Encoding y Dummy Coding

Con la codificación one-hot, el término de intersección (intercepto) representa la media global de la variable objetivo: 'Renta', y cada uno de los coeficientes lineales representa que tanto difiere la renta media de esa ciudad respecto a la media global.

Ahora entrenemos el mismo modelo con dummy coding

Con codificación dummy, el coeficiente de sesgo (intercepto) representa el valor medio de la variable $y$ para la categoría de referencia, que en el ejemplo es la ciudad NYC. El coeficiente para la *i*-ésima característica es igual a la diferencia entre el valor medio de la respuesta para la i-ésima categoría y la media de la categoría de referencia.

### Dummy Coding

In [40]:
col = 'drinks'

In [41]:
from sklearn.preprocessing import OneHotEncoder as OneHotEncoder_sk 

In [42]:
dummy_enc = OneHotEncoder_sk(drop='first',sparse=False)

In [43]:
dummy_enc.fit_transform(df[[col]])

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.]])

In [44]:
dummy_enc.categories_

[array(['desperately', 'drinks_missing', 'not_at_all', 'often', 'rarely',
        'socially', 'very_often'], dtype=object)]

In [45]:
dummy_df2 = pd.get_dummies(df[col], prefix='dr', drop_first=True)
dummy_df2.head()

,dr_drinks_missing,dr_not_at_all,dr_often,dr_rarely,dr_socially,dr_very_often
1,0,0,0,0,1,0
4,0,0,0,0,1,0
6,0,0,0,0,1,0
8,0,0,0,0,1,0
10,0,1,0,0,0,0


In [46]:
dummy_df2.drop_duplicates()

,dr_drinks_missing,dr_not_at_all,dr_often,dr_rarely,dr_socially,dr_very_often
1,0,0,0,0,1,0
10,0,1,0,0,0,0
19,0,0,0,1,0,0
41,0,0,1,0,0,0
93,1,0,0,0,0,0
97,0,0,0,0,0,1
549,0,0,0,0,0,0



### Sum (or Deviation) Coding

In [47]:
from category_encoders import SumEncoder

In [48]:
sum_enc = SumEncoder()

In [49]:
sum_enc.mapping[0]['mapping']

TypeError: ignored

Utilizando el ejemplo de prueba

In [ ]:
sum_enc = SumEncoder()
sum_df = sum_enc.fit_transform(df_test['City'])

In [ ]:
sum_df

In [ ]:
model = linear_model.LienarRegression()
model.fit(sum_df[['City_0','C']])


La codificación de efectos (sum coding) es muy similar a la codificación dummy, pero da como resultado modelos de regresión lineal que son aún más simples de interpretar.

En el ejemplo, vemos que el término de intersección representa la media global del la variable respuesta, y los coeficientes individuales indican cuánto las medias de las categorías individuales difieren de la media global. (Esto se llama el efecto principal de la categoría o nivel, de ahí el nombre "codificación de efectos").

La codificación One-hot, encontró la misma intersección y coeficientes, pero en ese caso hay coeficientes lineales para cada ciudad. En la codificación de efecto, ningun feature representa la categoría de referencia, ***por lo que el efecto de la categoría de referencia debe calcularse por separado como la suma negativa de los coeficientes de todas las demás categorías.***

In [ ]:
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')
print(f'Negative sum of coaefficients:{np.sum(model.coef_*-1)}')


In [ ]:
print('Global mean: ', df_test['Rent'].mean())
print(df_test.groupby('City').mean())

Restulados de la regresión lineal con One-hot encoding

Coefficients: $[ 166.66666667,   666.66666667,  -833.33333333]$

Intercept: $3333.3333333333335$

### Backward Difference Coding

In [ ]:
from category_encoders import BackwardDifferenceEncoder

In [ ]:
back_diff_enc = BackwardDifferenceEncoder()

In [ ]:
back_diff_enc.fit_transform(df[[col]])

In [ ]:
back_diff_enc.mapping[0]['mapping']

Utilizando el ejemplo de prueba

In [ ]:
back_diff_enc = BackwardDifferenceEncoder()
back_diff_df = back_diff_enc.fit_transform(df_test['City'])
back_diff_df['Rent'] = df_test['Rent']
back_diff_df


In [ ]:
back_diff_enc.mapping[0]['mapping']

In [ ]:
back_diff_enc.ordinal_encoder.category_mapping[0]['mapping']

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()
model.fit(back_diff_df[['City_0','City_1']], back_diff_df['Rent'])

La codificación de diferencia en reversa es útil para codificar variables ordinales.

En el ejemplo, vemos que el término de intersección representa la media global de la variable respuesta, y los coeficientes individuales indican cuánto las medias de las categorías individuales difieren de la media de la categoría inmediatamente anterior.

In [ ]:
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

In [ ]:
print('Global mean: ',df_test['Rent'].mean())
print(df_test.groupby('City').mean())

### Helmert Coding

In [ ]:
from category_encoders import HelmertEncoder  

In [ ]:
helm_enc = HelmertEncoder()

In [ ]:
helm_enc.fit_transform(df[[col]])

In [ ]:
helm_enc.mapping[0]['mapping']

Utilizando el ejemplo de prueba

In [ ]:
helm_enc = HelmertEncoder()
helm_df = helm_enc.fit_transform(df_test['City'])
helm_df['Rent']=df_test['Rent']
helm_df

In [ ]:
helm_enc.mapping[0]['mapping']

In [ ]:
model = linear_model.LinearRegression()
model.fit(helm_df[['City_0','City_1']],helm_df['Rent'])

In [ ]:
print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')

In [ ]:
print('Global mean: ', df_test['Rent'].mean())
print(df_test.groupby('City').mean())

In [ ]:
df_test[df_test['City']!='NYC']['Rent'].mean()

#Supervised Encoders

In [ ]:
col_cat = 'drinks'
col_num = 'essay_length'

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_enc = LabelEncoder()
df['Class_enc'] = label_enc.fit_transform(df['Class'])

In [ ]:
df.head()

## Target Encoder

In [ ]:
from category_encoders import TargetEncoder

In [ ]:
target_enc = TargetEncoder()
target_enc.fit_transform(df[col_cat],df['Class_enc'])

In [ ]:
target_enc.mapping

In [ ]:
target_enc = TargetEncoder()
target_enc.fit_transform(df[col_cat],df[col_num])

In [ ]:
target_enc.mapping

## Leave-one-out Encoder

In [ ]:
from category_encoders import LeaveOneOutEncoder

In [ ]:
lou_enc = LeaveOneOutEncoder(sigma=0.05)
lou_enc.fit_transform(df[col_cat],df['Class_enc'])

In [ ]:
lou_enc.mapping

In [ ]:
lou_enc = LeaveOneOutEncoder(sigma=0.05)
lou_enc.fit_transform(df[col_cat],df['essay_length'])

In [ ]:
lou_enc.mapping

## M-estimate Encoder

In [ ]:
from category_encoders import MEstimateEncoder

In [ ]:
mest_enc = MEstimateEncoder(m=10)
mest_enc.fit_transform(df[col_cat],df['Class_enc'])

In [ ]:
mest_enc.mapping

## Weight of Evidence Encoder

In [ ]:
from category_encoders import WOEEncoder 

In [ ]:
woe_enc = WOEEncoder()
woe_enc.fit_transform(df[col_cat],df['Class_enc'])

In [ ]:
woe_enc.mapping